In [1]:
import torch 
import time 

import pandas as pd
import itertools
from sklearn.model_selection import KFold

import torch.nn as nn
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from gensim.models.phrases import Phrases
from gensim.models import Word2Vec

from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.evaluation import classification_inference
from src.metric import  multi_cls_metrics,multi_cls_log
from src.dataset.tokenizer import GensimTokenizer
from src.preprocess.augment import *

from iflytek_app.dataset import MixDataset
from iflytek_app.models import TextcnnMixup
from iflytek_app.process import train_process, test_process, result_process, kfold_inference


device = get_torch_device()
set_seed()

No GPU available, using the CPU instead.


In [2]:
phraser= Phrases.load('./checkpoint/phrase_tokenizer')
c2v = GensimTokenizer( Word2Vec.load('./checkpoint/char_min1_win5_sg_d100'))
w2v = GensimTokenizer(Word2Vec.load('./checkpoint/phrase_min1_win5_sg_d100'), phraser)
w2v.init_vocab()
c2v.init_vocab()

df, label2idx = train_process()
test = test_process()

                id           l1           l2          len
count  4199.000000  4199.000000  4199.000000  4199.000000
mean   2099.000000     8.969278    37.087878    46.057156
std    1212.291219     4.576621    79.204914    79.332999
min       0.000000     2.000000     1.000000     4.000000
25%    1049.500000     5.000000     6.000000    15.000000
50%    2099.000000     8.000000    12.000000    22.000000
75%    3148.500000    12.000000    26.000000    36.000000
max    4198.000000    32.000000   946.000000   961.000000
{'14784131 14858934 14784131 14845064': 0, '14852788 14717848 15639958 15632020': 1, '14844856 14724258 14925237 14854807': 2, '14925756 15639967 14853254 14728639': 3, '14844593 14924945': 4, '15709098 14716590 14924703 14779559': 5, '14726332 14728344 14854542 14844591': 6, '14858934 15636660 15704193 14849963': 7, '15710359 14847407 14845602 14859696': 8, '14794687 14782344': 9, '15630486 15702410 14718849 15709093': 10, '15632285 15706536 14721977 14925219': 11, '147829

In [3]:
aug_samples = [] 

rand_del = WordDelete(w2v, max_sample=1, filters=())
rand_synon= W2vSynonymous(w2v, max_sample=1, filters=(),topn=3)
rand_swap = WordSwap(w2v, max_sample=1, filters=())


for index, row in enumerate(df.iterrows()):
    text = row[1]['description']
    name = row[1]['name']
    label = row[1]['label']
    for aug_text in rand_del.augment(name):
        aug_samples.append((index, aug_text.split(' '), text, label, 'word_random_delete'))
    for aug_text in rand_swap.augment(name):
        aug_samples.append((index, aug_text.split(' '), text, label, 'word_random_swap'))
    for aug_text in rand_synon.augment(text):
        aug_samples.append((index, aug_text.split(' '), text, label, 'word_random_synonymous'))
        
        
# rand_del = WordDelete(c2v, filters=())
# rand_synon= W2vSynonymous(c2v, filters=())
# rand_swap = WordSwap(c2v, filters=())

# for index, row in enumerate(df.iterrows()):
#     text = row[1]['description']
#     name = row[1]['name']
#     label = row[1]['label']
#     for aug_text in rand_del.augment(text):
#         aug_samples.append((index, name, aug_text.split(' '), label, 'char_random_delete'))
#     for aug_text in rand_swap.augment(text):
#         aug_samples.append((index, name, aug_text.split(' '), label, 'char_random_swap'))
#     for aug_text in rand_synon.augment(text):
#         aug_samples.append((index, name, aug_text.split(' '), label, 'char_random_synonymous'))
        
df_aug = pd.DataFrame(aug_samples, columns= ['index', 'name','description','label','augment'])    
df_aug['description'] = df_aug['description'].map(lambda x: list(itertools.chain(*[i.split('_') for i in x])))
df_aug['name'] = df_aug['name'].map(lambda x: list(itertools.chain(*[i.split('_') for i in x])))

In [12]:
log_steps = 10
save_steps = 20
tp = TrainParams(
    epoch_size=30,
    lr=1e-3,
    loss_fn=nn.CrossEntropyLoss(),
    max_seq_len=1000,
    batch_size=32,
    aug_batch_size=16,
    dropout_rate=0.5,
    label_size = len(label2idx),
    vocab_size = w2v.vocab_size,
    embedding_dim = w2v.embedding_size + c2v.embedding_size,
    embedding1=c2v.embedding, 
    embedding2 =w2v.embedding,
    num_train_steps=int(df.shape[0]/5 *4), 
    filter_size=70,
    kernel_size_list = [2,3,4,5],
    hidden_size = 150,
    early_stop_params = {
        'monitor':'f1_micro',
        'mode':'max',
        'min_delta': 0,
        'patience':5,
        'verbose':False
    },
    scheduler_params={'mode': 'max',
                     'factor': 0.5,
                     'patience': 2,
                     'verbose': True,
                     'threshold':0.0001,
                     'threshold_mode':'rel',
                     'cooldown':0,
                     'min_lr':1e-6},
    mixup_alpha =0.5
)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=24)
for fold,(train_index, valid_index) in enumerate(kf.split(df)):
    train, valid = df.iloc[train_index], df.iloc[valid_index]
    aug = df_aug.loc[df_aug['index'].isin(train_index),:] #避免数据泄露只使用train部分的增强数据
    train_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx, 
                               train['name'].values, train['description'].values, train['label'].values)
    valid_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx, 
                               valid['name'].values, valid['description'].values, valid['label'].values)
    train_sampler = RandomSampler(train_dataset)
    valid_sampler = SequentialSampler(valid_dataset)
    train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
    valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=tp.batch_size)
    
    aug_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx, 
               aug['name'].values, aug['description'].values, aug['label'].values)
    aug_sampler = RandomSampler(aug_dataset)
    aug_loader = DataLoader(aug_dataset, sampler=aug_sampler, batch_size=tp.aug_batch_size)
    
    CKPT = './checkpoint/textcnn_aug_mixup/k{}'.format(fold)
    saver = ModelSave(CKPT, continue_train=False)
    es = EarlyStop(**tp.early_stop_params)
    global_step = 0
    saver.init()
    tb = SummaryWriter(CKPT)
    model = TextcnnMixup(tp)
    optimizer, scheduler = model.get_optimizer()
    
    aug_iter = iter(aug_loader)
    for epoch_i in range(tp['epoch_size']):
        try:
            aug_batch = next(aug_iter)
        except:
            aug_iter = iter(aug_loader)
            aug_batch = next(aug_iter)

        if global_step==1:
            print(model)
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
        print("-"*60)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()
        total_loss, batch_loss, batch_counts = 0, 0, 0

        model.train()
        aug_iter = iter(aug_loader)
        for step, batch in enumerate(train_loader):
            aug_batch = next(aug_iter)
            global_step +=1
            batch_counts +=1

            #Forward propogate
            model.zero_grad()
            feature = {k:torch.cat([v, aug_batch[k]],dim=0).to(device) for k,v in batch.items()}
            logits = model(feature)
            loss = model.compute_loss(feature, logits)
            batch_loss += loss.item()
            total_loss += loss.item()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()

            # Log steps for train loss logging
            if (step % log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
                time_elapsed = time.time() - t0_batch
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
                tb.add_scalar('loss/batch_train', batch_loss / batch_counts, global_step=global_step)
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

            # Save steps for ckpt saving and dev evaluation
            if (step % save_steps == 0 and step != 0) or (step == len(train_loader) - 1):
                val_metrics = multi_cls_metrics(model, valid_loader, device)
                for key, val in val_metrics.items():
                    tb.add_scalar(f'metric/{key}', val, global_step=global_step)
                avg_train_loss = total_loss / step
                tb.add_scalars('loss/train_valid',{'train': avg_train_loss,
                                                    'valid': val_metrics['val_loss']}, global_step=global_step)
                saver(total_loss / step, val_metrics['val_loss'], epoch_i, global_step, model, optimizer, scheduler)

        # On Epoch End: calcualte train & valid loss and log overall metrics
        time_elapsed = time.time() - t0_epoch
        val_metrics = multi_cls_metrics(model, valid_loader, device)
        avg_train_loss = total_loss / step
        scheduler.step(val_metrics['f1_micro'])
        print("-"*70)
        print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
        multi_cls_log(epoch_i, val_metrics)
        print("\n")
        if es.check(val_metrics):
            break 

./checkpoint/textcnn_aug_mixup/k0 model cleaned
 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   10    |   4.028800   |     -     |   13.05  
   1    |   20    |   2.665085   |     -     |   12.24  
   1    |   30    |   2.693902   |     -     |   22.59  
   1    |   40    |   2.462389   |     -     |   13.94  
   1    |   50    |   2.137273   |     -     |   20.95  
   1    |   60    |   2.130582   |     -     |   11.84  
   1    |   70    |   2.041021   |     -     |   19.01  
   1    |   80    |   1.781639   |     -     |   11.19  
   1    |   90    |   1.763078   |     -     |   18.52  
   1    |   100   |   1.605632   |     -     |   10.69  
   1    |   104   |   1.682020   |     -     |   12.01  
----------------------------------------------------------------------
   1    |    -    |   2.344720   |  1.666616  |  173.12  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Reca

   7    |   10    |   1.211278   |     -     |   12.51  
   7    |   20    |   1.247448   |     -     |   11.41  
   7    |   30    |   0.296465   |     -     |   15.87  
   7    |   40    |   0.230450   |     -     |   9.89   
   7    |   50    |   0.157613   |     -     |   16.06  
   7    |   60    |   0.175851   |     -     |   9.83   
   7    |   70    |   0.186275   |     -     |   15.68  
   7    |   80    |   0.185580   |     -     |   9.65   
   7    |   90    |   0.142007   |     -     |   15.97  
   7    |   100   |   0.133250   |     -     |   9.76   
   7    |   104   |   0.075027   |     -     |   9.98   
----------------------------------------------------------------------
   7    |    -    |   0.395900   |  1.173567  |  142.69  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   7    |  56.663%  |  89.489%  |  64.960%  |     67.066%     

  13    |   10    |   1.431689   |     -     |   12.70  
  13    |   20    |   0.851295   |     -     |   11.09  
  13    |   30    |   0.150277   |     -     |   15.53  
  13    |   40    |   0.108037   |     -     |   9.89   
  13    |   50    |   0.056476   |     -     |   16.08  
  13    |   60    |   0.064657   |     -     |   9.61   
  13    |   70    |   0.088334   |     -     |   16.03  
  13    |   80    |   0.080300   |     -     |   9.69   
  13    |   90    |   0.052473   |     -     |   16.14  
  13    |   100   |   0.048778   |     -     |   9.70   
  13    |   104   |   0.029971   |     -     |   9.95   
----------------------------------------------------------------------
  13    |    -    |   0.296872   |  1.253982  |  142.70  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
  13    |  57.290%  |  89.461%  |  64.224%  |     68.123%     

./checkpoint/textcnn_aug_mixup/k1 model cleaned
 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   10    |   3.731723   |     -     |   12.58  
   1    |   20    |   2.557498   |     -     |   11.38  
   1    |   30    |   2.663686   |     -     |   16.00  
   1    |   40    |   2.415569   |     -     |   9.89   
   1    |   50    |   2.199689   |     -     |   15.75  
   1    |   60    |   2.070654   |     -     |   9.78   
   1    |   70    |   1.829724   |     -     |   16.91  
   1    |   80    |   1.885645   |     -     |   9.94   
   1    |   90    |   1.685666   |     -     |   15.99  
   1    |   100   |   1.604464   |     -     |   9.76   
   1    |   104   |   1.598993   |     -     |   10.00  
----------------------------------------------------------------------
   1    |    -    |   2.274720   |  1.563092  |  143.99  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Reca

   7    |   10    |   1.194595   |     -     |   12.68  
   7    |   20    |   1.285147   |     -     |   11.06  
   7    |   30    |   0.320399   |     -     |   15.95  
   7    |   40    |   0.230895   |     -     |   9.90   
   7    |   50    |   0.215797   |     -     |   16.11  
   7    |   60    |   0.231140   |     -     |   9.78   
   7    |   70    |   0.147844   |     -     |   16.01  
   7    |   80    |   0.199201   |     -     |   9.87   
   7    |   90    |   0.203947   |     -     |   16.04  
   7    |   100   |   0.190530   |     -     |   9.69   
   7    |   104   |   0.201534   |     -     |   9.90   
Epoch     7: reducing learning rate of group 0 to 5.0000e-04.
----------------------------------------------------------------------
   7    |    -    |   0.424958   |  1.031078  |  143.37  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------


  13    |   10    |   1.183747   |     -     |   12.53  
  13    |   20    |   0.920805   |     -     |   11.25  
  13    |   30    |   0.112506   |     -     |   16.03  
  13    |   40    |   0.105602   |     -     |   9.73   
  13    |   50    |   0.081937   |     -     |   15.89  
  13    |   60    |   0.071443   |     -     |   9.76   
  13    |   70    |   0.097475   |     -     |   15.66  
  13    |   80    |   0.054658   |     -     |   9.74   
  13    |   90    |   0.070167   |     -     |   15.72  
  13    |   100   |   0.055857   |     -     |   9.77   
  13    |   104   |   0.060764   |     -     |   10.00  
Epoch    13: reducing learning rate of group 0 to 2.5000e-04.
----------------------------------------------------------------------
  13    |    -    |   0.278546   |  0.947485  |  142.40  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------


In [9]:
test = test_process()
test_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx,
                                         test['name'].values, test['description'].values)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=tp.batch_size)

In [11]:
result = kfold_inference(test_loader, tp, TextcnnMixup, './checkpoint/textcnn_aug', 5, device)
result['pred'] = result['pred_avg']
result_process(result, label2idx, './submit/textcnn_aug_5fold_avg.csv')